In [1]:
%%init_spark
launcher.jars = ["libs/kensu-dam-spark-collector-0.17.2_spark-3.0.1.jar","libs/sdk_2.12.jar"]
launcher.conf.set("spark.sql.shuffle.partitions", "1")

In [2]:
val week = "week1"
implicit val ch = new io.kensu.dodd.sdk.ConnectHelper(s"./conf/application1-$week.properties")
val dataFolder = "data"+ch.properties("dam.activity.user").toString
io.kensu.third.integration.TimeUtils.setMockedTime(ch.properties("mocked.timestamp").asInstanceOf[Long], false)
io.kensu.dam.lineage.spark.lineage.Implicits.SparkSessionDAMWrapper(spark).track(ch.properties.get("dam.ingestion.url").map(_.toString), None)(ch.properties.toList:_*)
val sdk = new io.kensu.dodd.sdk.SDK(ch.properties("kensu.sdk.url").toString, ch.properties("kensu.pat").toString)

Intitializing Scala interpreter ...

Spark Web UI available at http://jupyter-kensuio-2doss-2ddodd-2dworkshop-2d4tngjcu1:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1636383299915)
SparkSession available as 'spark'


maybeDamFileDebugLevel: Some(INFO)
Activating DAM debug logging to file: debug.log
Done activating DAM debug logging to a file
Creating a new DAM scala client actor-system
Done creating a new DAM scala client actor-system.


week: String = week1
ch: io.kensu.dodd.sdk.ConnectHelper = io.kensu.dodd.sdk.ConnectHelper@2f1948f1
dataFolder: String = "dataSammy "
sdk: io.kensu.dodd.sdk.SDK = io.kensu.dodd.sdk.SDK@6661f287


In [3]:
val allRules = sdk.getRules()

allRules: org.json4s.JObject = JObject(List((data,JArray(List(JObject(List((uuid,JString(02ab03b4-e3d1-48d1-ae44-bde4c614011c)), (environment,JString(Production)), (fieldName,JString()), (functionName,JString(Frequency)), (entityType,JString(DATA_STATS)), (arguments,JObject(List((timeLapse,JInt(1)), (expiresAt,JInt(1615849207200)), (timeLapseUnit,JString(Days))))), (setBy,JString(USER)), (isDeleted,JBool(false)), (isTracked,JBool(true)), (isLogical,JBool(true)), (creationTimestamp,JString(2021-09-20T16:02:46.497Z)), (datasource,JObject(List((id,JString(467747eb-4da8-4fab-80e1-47c4ede39ac1)), (name,JString(Recommendations Table))))), (project,JObject(List((id,JString(k-0c316854fa0c531a708e63646c773646b86566799fb6ffd1090e0eeaea5ae15f)), (name,JString(Recommendation Pipeline))))), (process...


In [46]:
val rule = sdk.range(Some(3),None)

rule: sdk.Rule = (Range,Map(minVal -> 3))


In [45]:
sdk.addRule("trial","id.nullrows",("Range", Map("minVal" -> 0, "maxVal" -> 1000)))

In [20]:
import _root_.io.kensu.dam.model._

import _root_.io.kensu.dam.model._


In [24]:
val env: String = ch.properties("dam.activity.environment").toString

env: String = workshop


In [37]:
projectId

res12: String = k-013aba01970ed8c2403612ac8094bf79fd04415890777d6d9d7e13238fe6834c


In [33]:
import org.json4s._
import org.json4s._

import org.json4s._
import org.json4s._


In [34]:
val ldsId = ((lineages \ "data" \ "nodes" \ "datasource").filter(_ \ "name" == JString("trial")).head \ "id").values.toString

ldsId: String = logical-data-source-trial


In [35]:
val lineageIds = (lineages \ "data" \ "links").filter(l => l \ "sourceDatasourceId" == JString(ldsId) || l \ "targetDatasourceId" == JString(ldsId)).map(_ \ "lineage" \ "id").map(_.toString)

lineageIds: List[String] = List(JString(k-8baa873c8cc710c66141363daae4d2125d8f38eac2c428b4644870b50f3a8d09), JString(k-fb14d6038c0aff5d96dbf786327c618018366d4fe0a6d42b0ad9178a26f6cdd1), JString(k-a030ab34da8606f70b7a6cd3e2c0adeb170e39c43bf0ac9b9d2f1ad601f9d9f7))


In [19]:
import _root_.io.kensu.dam.model._
val projectId:String = ProjectPK(ch.properties("dam.activity.projects").asInstanceOf[Array[String]].head).guid
val processId: String = ProcessPK(ch.properties("dam.activity.explicit.process.name").toString).guid
val env: String = ch.properties("dam.activity.environment").toString
val codeVersion: String = CodeVersionPK(ch.properties("dam.activity.code.version").toString, CodeBaseRef(byPK=None, byGUID=Some(CodeBasePK(ch.properties("dam.activity.code.repository").toString).guid))).guid
val lineages = sdk.getLineagesInProject(projectId, processId, env, codeVersion)
val ldsId = ((lineages \ "data" \ "nodes" \ "datasource").filter(_ \ "name" == JString(dsName)).head \ "id").values.toString
val lineageIds = (lineages \ "data" \ "links").filter(l => l \ "sourceDatasourceId" == JString(ldsId) || l \ "targetDatasourceId" == JString(ldsId)).map(_ \ "lineage" \ "id").map(_.toString)
lineageIds.foreach { lineageId =>
    sdk.createRule(ldsId, lineageId, projectId, processId, env, fieldName, rule)
}

<console>: 33: error: not found: value JString

In [42]:
sdk.range(Some(3),Some(4))

res15: sdk.Rule = (Range,Map(minVal -> 3, maxVal -> 4))


In [ ]:
def dsRules = (sdk.getRulesForDs(
    "logical-data-source-joined_data",
    "k-2a8a077a91f490348a95cbcb6d524e7c232ee4abe935cc68d25f01ea5980cdac",
    "k-ddc546028d6968a8b747d2b417d79c6d2ef650911422aa40a3283ea1492e4d57",
    "workshop"
) \ "data" \ "predicates").asInstanceOf[org.json4s.JArray]
dsRules

In [ ]:
val r = dsRules
if (r.values.size != 0) {
    // delete has issues...
    // sdk.deleteRule((r \\ "uuid").values.toString)
}

In [ ]:
dsRules

In [ ]:
sdk.createRule(
    "logical-data-source-joined_data",
    "k-2a8a077a91f490348a95cbcb6d524e7c232ee4abe935cc68d25f01ea5980cdac",
    "k-ddc546028d6968a8b747d2b417d79c6d2ef650911422aa40a3283ea1492e4d57",
    "k-ddb97a5623d591997c7e8b83caa660f2509958106614f2d3e5905d726f1a94d5",
    "workshop",
    "price.mean",
    ("Range", Map("minVal" -> 0, "maxVal" -> 1000))
)

In [ ]:
dsRules

In [ ]:
sdk.updateRule(
    (dsRules \\ "uuid").values.toString,
    ("Range", Map("minVal" -> 100, "maxVal" -> 1000))
)

In [ ]:
dsRules